In [ ]:
import re

In [ ]:
def find_wrong_indices(file_path):
    with open(file_path, 'r') as file:
        log_content = file.read()
    matches = re.findall(r'index: (\d+)[\s|\n|\|]+\|\sWrong!', log_content, re.DOTALL)
    wrong_indices = [int(match) for match in matches]
    return wrong_indices

file_path = '/home/yl3427/cylab/selfCorrectionAgent/result_mixtral.log'
# '/home/yl3427/cylab/selfCorrectionAgent/result_mixtral.log'
# '/home/yl3427/cylab/selfCorrectionAgent/result_llama3.log'

wrong_indices = find_wrong_indices(file_path)

print("Indices with 'Wrong!':", wrong_indices)


In [ ]:
len(wrong_indices)

In [ ]:
indices = []
for i in range(len(wrong_indices)-1):
    if wrong_indices[i] > wrong_indices[i+1]:
        print(i)
        print(wrong_indices[i: i+2])
        indices.append(i)


In [ ]:
idx1 = indices[0]+1
idx2 = indices[1]+1
base = set(wrong_indices[:idx1])
always = set(wrong_indices[idx1:idx2])
condition = set(wrong_indices[idx2:])

In [ ]:
len(base)

In [ ]:
len(base & always), base & always # 49

In [ ]:
len(base - always), base - always 

In [ ]:
len(always - base),always - base # 15

In [ ]:
import pandas as pd

brca_report = pd.read_csv("/secure/shared_data/rag_tnm_results/summary/5_folds_summary/brca_df.csv")
sample_reports = brca_report.iloc[:500, :]

sample_reports

In [ ]:
for i in sorted(always - base):
    print(f"T{sample_reports.iloc[i].t+1}")

In [ ]:
def find_similarity(file_path):
    with open(file_path, 'r') as file:
        log_content = file.read()
    matches = re.findall(r'similarity: (\d+)', log_content, re.DOTALL)
    scores = [int(match) for match in matches]
    return scores

file_path = '/home/yl3427/cylab/selfCorrectionAgent/result_llama3.log'

# '/home/yl3427/cylab/selfCorrectionAgent/result_mixtral.log'
# '/home/yl3427/cylab/selfCorrectionAgent/result_llama3.log'

similarity_scores = find_similarity(file_path)

print(similarity_scores)


In [ ]:
len(similarity_scores)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

index = range(0, 500)


# Plotting
plt.figure(figsize=(10, 5))
plt.plot(index, similarity_scores)
plt.xlabel('index of report')
plt.ylabel('similiarity between created rules and the existing rules (memory)')
plt.xlim(0, 500)
plt.xticks(np.arange(0, 501, 25))  # Adjust the interval as needed
plt.legend()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

index = range(0, 80)


# Plotting
plt.figure(figsize=(10, 5))
plt.plot(index, similarity_scores[:80])
plt.xlabel('index of report')
plt.ylabel('similiarity between created rules and the existing rules (memory)')
plt.xlim(0, 80)
plt.xticks(np.arange(0, 80, 5))  # Adjust the interval as needed
plt.legend()
plt.show()


In [ ]:
import ast

In [ ]:
def find_memory(file_path):
    with open(file_path, 'r') as file:
        log_content = file.read()
    matches = re.findall(r" memory: (\[.*?\])", log_content, re.DOTALL)
    wrong_indices = [ast.literal_eval(match) for match in matches]
    return wrong_indices

file_path = '/home/yl3427/cylab/selfCorrectionAgent/memory.txt'


wrong_indices = find_memory(file_path)

print(len(wrong_indices))
print(wrong_indices)

In [1]:
import pandas as pd
import re
from src.metrics import relax_n03_performance_report, n03_performance_report

In [2]:
print("zshot")
df = pd.read_csv("result/n03/brca_df_zs_n03.csv")
relax_n03_performance_report(df, "zs_n_ans_str")
print()

for size in range(10, 101, 5):
    print(f"conditional memory, test size: {size}")
    df = pd.read_csv(f"/home/yl3427/cylab/selfCorrectionAgent/result/n03/saturation_test_result_{size}.csv")
    n03_performance_report(df, "cmem_n_ans_str")
    print()

zshot
              precision    recall  f1-score   support

          N0       0.95      0.94      0.95       316
          N1       0.86      0.87      0.87       300
          N2       0.63      0.71      0.67       110
          N3       0.90      0.76      0.82        74

    accuracy                           0.86       800
   macro avg       0.84      0.82      0.83       800
weighted avg       0.87      0.86      0.87       800


conditional memory, test size: 10
              precision    recall  f1-score   support

          N0       0.94      0.95      0.95       315
          N1       0.86      0.90      0.88       295
          N2       0.68      0.74      0.71       108
          N3       1.00      0.67      0.80        72

    accuracy                           0.88       790
   macro avg       0.87      0.81      0.83       790
weighted avg       0.88      0.88      0.88       790


conditional memory, test size: 15


/home/yl3427/miniconda3/envs/selfCorrection/lib/python3.9/site-packages/sklearn/metrics/_classification.py:2628: UserWarning: labels size, 4, does not match size of target_names, 5
  warnings.warn(


              precision    recall  f1-score   support

          N0       0.94      0.97      0.95       306
          N1       0.89      0.84      0.87       290
          N2       0.67      0.81      0.73       108
          N3       0.96      0.75      0.84        71

    accuracy                           0.88       775
   macro avg       0.87      0.84      0.85       775
weighted avg       0.89      0.88      0.88       775


conditional memory, test size: 20
              precision    recall  f1-score   support

          N0       0.95      0.97      0.96       309
          N1       0.90      0.84      0.87       292
          N2       0.65      0.77      0.70       105
          N3       0.90      0.77      0.83        71

    accuracy                           0.88       777
   macro avg       0.85      0.84      0.84       777
weighted avg       0.89      0.88      0.88       777


conditional memory, test size: 25
              precision    recall  f1-score   support

     

In [ ]:
for size in range(10, 101, 10):
    df = pd.read_csv(f"/home/yl3427/cylab/selfCorrectionAgent/result/saturation_test_result_{size}.csv")
    n03_performance_report(df, "cmem_n_ans_str")

In [ ]:
df[df["cmem_n_ans_str"].isnull()]

In [ ]:
# load dataframe from csv file
df = pd.read_csv("saturation_train_result_10.csv")
n03_performance_report(df, "mem_n_ans_str")
df = pd.read_csv("saturation_train_result_20.csv")
n03_performance_report(df, "mem_n_ans_str")
df = pd.read_csv("saturation_train_result_30.csv")
n03_performance_report(df, "mem_n_ans_str")